In [42]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import matplotlib.pyplot as plt
import rl.models as rla
import config

In [ ]:
# def print_trade_length(data:pd.DataFrame):
#     df = data['action'].eq(data['action'].shift()).cumsum()
        


#     average_neutral = (df[0]*df.index).sum()/df[0].sum()
#     average_long = (df[1]*df.index).sum()/df[1].sum() 
#     average_short = (df[2]*df.index).sum()/df[2].sum()
#     average_all = ((df[0] + df[1] + df[2])*df.index).sum()/(df[0]+df[1]+df[2]).sum()
#     print("Average length 1:{} | 2:{} | 0:{} | All:{}".format(
#         round(average_long,2),round(average_short,2),round(average_neutral,2),round(average_all,2)))
#     print(df)
#     return df

In [ ]:
fm = fma.FeatureManager(target_col="trade_signal")
fm.import_trading_data(symbol="BTCUSDT",trade_timeframe="4h")
fm.import_macro_data(symbol="BTCUSDT",macro_timeframe="1d")
fm.import_super_data(symbol="BTCUSDT",super_timeframe="1w")

FEATURES = [
    "sma_3_10",
    "sma_7_30",
    "sma_14_50",
    "rsi14",
    "rsi30",
    "obv",
    "mfi14",
]
fm.build_features(lags = 1,macro_lags=1,super_lags=1,features=FEATURES,macro_features=FEATURES,super_features=FEATURES)


In [ ]:
train = fm.df.iloc[-4000:-2000]
trade = fm.df.iloc[-2000:]

In [44]:
importlib.reload(rla)
importlib.reload(env)

env_kwargs = config.TRADE_ENV_PARAMETER
state_space = 3 + len(fm.cols)

train_env = env.CryptoTradingEnv(
    df = train, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

trade_env = env.CryptoTradingEnv(
    df = trade, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [ ]:
agent = rla.DRLTradeAgent(train_env)

model_ppo = agent.get_model(model_name="ppo",seed=100)
model_a2c = agent.get_model(model_name="a2c",seed=100)
model_ars = agent.get_model(model_name="ars",seed=100)
model_trpo = agent.get_model(model_name="trpo",seed=100)

In [ ]:
selected_model = model_ppo

In [ ]:
selected_model = agent.train_model(selected_model,total_timesteps=30_000)
selected_model.num_timesteps

In [ ]:
agent.predict(model=selected_model,environment=trade_env,render = False)
result = agent.make_result_data()
print(f"Number of trades {agent.trade_number}")
agent.plot_multiple(result)

In [ ]:
model = agent.load_model("ppo","saved_model/model_ppo.zip")

In [41]:
model.save("../saved_model/model_ppo.zip")

In [ ]:
agent.predict(model=model,environment=trade_env,render=False)

In [ ]:
agent.plot_multiple(agent.make_result_data())